In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
from tqdm import tqdm

In [3]:
def scrap():
    authors = [] #store the song names
    detail = []
    scraper = cloudscraper.create_scraper()
    for j in tqdm(range(1, 51187)): #51186
        try:
            url = "https://www.yes24.com/24/AuthorFile/Author/"+str(j)
            r= scraper.get(url)
            soup=BeautifulSoup(r.text,"html.parser")

            #start extracting info
            #all info are inside <tbody> </tbody>
            #inside <tbody>, EACH song is encapsulated in a <tr></tr>
            #inside <tr>,  the song name and singer is encapsulated inside <td class="chart-table-track">: <strong> encapsulates the song name
                                                               #   <span> encapsulates the singer
            #inside <tr>,  <td class_="chart-table-streams"> stores the streams 
            authors.append(soup.select_one('#wrapperContent span:nth-child(1)').text)
            whole_table = soup.select('table[width="150"] td[height="20"]')

            ls = []
            for i in whole_table:
                ls.append(i.text)
            detail.append(ls)

        except:
            authors.append('')
            detail.append(['','','','',''])
        
    return authors,detail


##########################################
#put the result into a csv file
##########################################
# data = scrap(weeks)
# df = pd.DataFrame(np.column_stack(data),columns=['date','song','singer','streams','url_list'])
# df.to_csv('2022_kr.csv',index=False)
# print("Done!")


In [4]:
data = scrap()

100%|██████████████████████████████████████████████████████████████████████████| 51186/51186 [1:41:55<00:00,  8.37it/s]


## 데이터 전처리

In [6]:
df = pd.DataFrame(np.column_stack(data),columns=['authors','detail'])

C:\Users\jsyu\anaconda3\envs\bworm\lib\site-packages\numpy\lib\shape_base.py:652: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(v)


In [16]:
df = df[df.authors!='']

In [35]:
df['born'] = df.detail.apply(lambda x : x[0] if x[0].startswith("출생 :") else '출생 : ')

C:\Users\jsyu\AppData\Local\Temp\ipykernel_16112\99541381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['born'] = df.detail.apply(lambda x : x[0] if x[0].startswith("출생 :") else '출생 : ')


In [46]:
df['death'] = df.detail.apply(lambda x : x[0] if x[0].startswith("사망 :") else ( x[1] if x[1].startswith("사망 :") else '사망 : '))

C:\Users\jsyu\AppData\Local\Temp\ipykernel_16112\3989778922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['death'] = df.detail.apply(lambda x : x[0] if x[0].startswith("사망 :") else ( x[1] if x[1].startswith("사망 :") else '사망 : '))


In [64]:
df['born_area'] = df.detail.apply(lambda x : (x[0] if x[0].startswith("출생지 :") else ( x[1] if x[1].startswith("출생지 :") else ( x[2] if x[2].startswith("출생지 :") else '출생지 : '))))

C:\Users\jsyu\AppData\Local\Temp\ipykernel_16112\3428908940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['born_area'] = df.detail.apply(lambda x : (x[0] if x[0].startswith("출생지 :") else ( x[1] if x[1].startswith("출생지 :") else ( x[2] if x[2].startswith("출생지 :") else '출생지 : '))))


In [69]:
df['sex'] = df.detail.apply(lambda x : (x[1] if x[1].startswith("성별 :") else '성별 :' )if len(x)<=3 else (x[2] if x[2].startswith("성별 :") else ( x[3] if x[3].startswith("성별 :") else ( x[4] if x[4].startswith("성별 :") else '성별 : '))))

C:\Users\jsyu\AppData\Local\Temp\ipykernel_16112\914231354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'] = df.detail.apply(lambda x : (x[1] if x[1].startswith("성별 :") else '성별 :' )if len(x)<=3 else (x[2] if x[2].startswith("성별 :") else ( x[3] if x[3].startswith("성별 :") else ( x[4] if x[4].startswith("성별 :") else '성별 : '))))


In [71]:
df['job'] = df.detail.apply(lambda x : (x[2] if x[2].startswith("직업 :") else '직업 :' )if len(x)<=3 else (x[2] if x[2].startswith("직업 :") else ( x[3] if x[3].startswith("직업 :") else ( x[4] if x[4].startswith("직업 :") else '직업 : '))))

C:\Users\jsyu\AppData\Local\Temp\ipykernel_16112\2620012714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job'] = df.detail.apply(lambda x : (x[2] if x[2].startswith("직업 :") else '직업 :' )if len(x)<=3 else (x[2] if x[2].startswith("직업 :") else ( x[3] if x[3].startswith("직업 :") else ( x[4] if x[4].startswith("직업 :") else '직업 : '))))


In [72]:
df

,authors,detail,born,death,born_area,sex,job
0,나예리,"[출생지 : , 성별 : 여자, 직업 : 만화가]",출생 :,사망 :,출생지 :,성별 : 여자,직업 : 만화가
1,이문열,"[출생 : 1948년 06월 24일, 출생지 : 서울특별시, 성별 : 남자, 직업 ...",출생 : 1948년 06월 24일,사망 :,출생지 : 서울특별시,성별 : 남자,직업 : 소설가
2,조애너 콜,"[출생 : 1944년 08월 11일, 사망 : 2020년 07월 12일, 출생지 :...",출생 : 1944년 08월 11일,사망 : 2020년 07월 12일,출생지 : 미국 뉴저지,성별 : 여자,직업 : 어린이 작가
4,이호백,"[출생 : 1962년 출생, 출생지 : 서울, 성별 : 남자, 직업 : 화가, 출판...",출생 : 1962년 출생,사망 :,출생지 : 서울,성별 : 남자,"직업 : 화가, 출판사대표"
5,권정생,"[출생 : 1937년 09월 10일, 사망 : 2007년 05월 17일, 출생지 :...",출생 : 1937년 09월 10일,사망 : 2007년 05월 17일,출생지 : 일본 도쿄,성별 : 남자,직업 : 아동문학가
...,...,...,...,...,...,...,...
51181,메인 스트릿,"[출생지 : , 성별 : , 직업 : ]",출생 :,사망 :,출생지 :,성별 :,직업 :
51182,로맨틱 카우치 (Romantic Couch),"[출생지 : , 성별 : , 직업 : ]",출생 :,사망 :,출생지 :,성별 :,직업 :
51183,한예슬,"[출생지 : , 성별 : , 직업 : ]",출생 :,사망 :,출생지 :,성별 :,직업 :
51184,오지호,"[출생 : 1976년 04월 14일, 출생지 : 한국, 성별 : , 직업 : 배우]",출생 : 1976년 04월 14일,사망 :,출생지 : 한국,성별 :,직업 : 배우


In [73]:
df.to_csv('author_detail.csv',index=False)